# Keras によるモデル開発 (ローカル環境)

Keras を利用したモデル学習を行います。手書き数字を区別する画像認識モデルを構築します。

## Azure Machine Learning Python SDK インポート

In [1]:
from azureml.core import Workspace, Dataset, Experiment
print(azureml.core.VERSION)

1.0.62


## Azure Machine Learning Workspace への接続
Azure Machine Learning service へ接続を行います。

In [2]:
subscription_id = '9c0f91b8-eb2f-484c-979c-15848c098a6b'
resource_group = 'dllab-test'
workspace_name = 'azureml'

workspace = Workspace(subscription_id, resource_group, workspace_name)

## 実験名の設定
過去の実験履歴をまとめるために名前をつけておきます。

In [3]:
experiment = Experiment(workspace = workspace, name = "keras_mnist_local")

## モデル学習コード

### ライブラリのインポート

In [4]:
from __future__ import print_function
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation
from keras.optimizers import SGD
from keras.utils import np_utils
import tensorflow as tf
#from keras.utils import plot_model
from tensorflow.python.keras.utils.vis_utils import plot_model
#from tensorflow.keras.vis_utils import plot_model

Using TensorFlow backend.


### ハイパーパラメータ設定

In [5]:
NB_EPOCH = 20
BATCH_SIZE = 32
NB_CLASSES = 10 #number of outputs = number of digits
OPTIMIZER = SGD() #SGD Optimizer
N_HIDDEN = 128
VALIDATION_SPLIT = 0.3 # Training and Validation data ratio
DROPOUT = 0.3 

In [18]:
class RunCallback(tf.keras.callbacks.Callback):
    def __init__(self, run):
        self.run = run
        
    def on_epoch_end(self, batch, logs={}):
        self.run.log(name="training_acc", value=float(logs.get('acc')))
        self.run.log(name="validation_acc", value=float(logs.get('val_acc')))

### 学習データの準備

In [19]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [20]:
RESHAPED = 784

In [21]:
X_train = X_train.reshape(60000,RESHAPED)
X_test = X_test.reshape(10000,RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [22]:
X_train /= 255
X_test /= 255

In [23]:
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

### モデル実装

In [24]:
model = Sequential()
model.add(Dense(N_HIDDEN, input_shape = (RESHAPED,)))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               100480    
_________________________________________________________________
activation_4 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_5 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1290      
__________

In [25]:
callbacks = list()

In [26]:
model.compile(loss ='categorical_crossentropy', optimizer = OPTIMIZER, metrics=['accuracy'])

### Azure Machine Learning service メトリック & ログ 記録開始

In [27]:
run = experiment.start_logging()

In [28]:
callbacks.append(RunCallback(run))

## モデル学習

In [29]:
model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE, 
          epochs=NB_EPOCH, 
          callbacks= callbacks,
          verbose = 1, 
          validation_split=VALIDATION_SPLIT)

Train on 42000 samples, validate on 18000 samples
Epoch 1/20
42000/42000 [==============================] - 7s 156us/step - loss: 1.0292 - acc: 0.6793 - val_loss: 0.4161 - val_acc: 0.8866
Epoch 2/20
42000/42000 [==============================] - 6s 138us/step - loss: 0.5022 - acc: 0.8502 - val_loss: 0.3105 - val_acc: 0.9110
Epoch 3/20
42000/42000 [==============================] - 6s 142us/step - loss: 0.4042 - acc: 0.8841 - val_loss: 0.2667 - val_acc: 0.9215
Epoch 4/20
42000/42000 [==============================] - 6s 147us/step - loss: 0.3498 - acc: 0.8986 - val_loss: 0.2377 - val_acc: 0.9293
Epoch 5/20
42000/42000 [==============================] - 6s 146us/step - loss: 0.3149 - acc: 0.9083 - val_loss: 0.2184 - val_acc: 0.9348
Epoch 6/20
42000/42000 [==============================] - 6s 143us/step - loss: 0.2890 - acc: 0.9160 - val_loss: 0.2039 - val_acc: 0.9380
Epoch 7/20
42000/42000 [==============================] - 6s 144us/step - loss: 0.2702 - acc: 0.9216 - val_loss: 0.1911 - 

In [30]:
score = model.evaluate(X_test, Y_test, verbose= 1)
print("Test score:", score[0])
print("Test accuracy" , score[1])
print("Loss value", score[0])
print("Accuracy value", score[1])

10000/10000 [==============================] - 0s 27us/step
Test score: 0.10981544711720198
Test accuracy 0.9651
Loss value 0.10981544711720198
Accuracy value 0.9651


## メトリックの記録

In [31]:
run.log("Loss", score[0])
run.log("Accuracy", score[1])
run.log("BATCH_SIZE", BATCH_SIZE)

## モデル保存

In [32]:
# !pip install pydot
# !pip install pydot-ng
# !pip install graphviz
# !sudo apt-get -y install graphviz

In [33]:
with run:
    #plot_model(model, to_file='./outputs/model.png')
    model.save_weights('./outputs/mnist_mlp_weights.h5')

In [34]:
run.complete()

In [35]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
keras_mnist_local,42ae17df-f545-410c-a69b-f13adfc29211,,Running,Link to Azure Portal,Link to Documentation
